In [1]:
import numpy as np
import pandas as pd
import datetime
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

## Подход 1: градиентный бустинг
1. Считайте таблицу с признаками из файла features.csv с помощью кода, приведенного выше. Удалите признаки, связанные с итогами матча (они помечены в описании данных как отсутствующие в тестовой выборке).

In [2]:
features = pd.read_csv('./features.csv', index_col='match_id')
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27.0,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16.0,2449,0,4,1974,3,63


Целевая переменная - radiant_win.  
Признаки, связанные с итогами матча:  
- `duration`: длительность
- `radiant_win`: 1, если победила команда Radiant, 0 — иначе
    - Состояние башен и барраков к концу матча (см. описание полей набора данных)
        - `tower_status_radiant`
        - `tower_status_dire`
        - `barracks_status_radiant`
        - `barracks_status_dire`

In [3]:
y = features['radiant_win']
drop_columns = ['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant',\
                'barracks_status_dire']
features = features.drop(drop_columns, axis='columns')
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


2. Проверьте выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. Много ли пропусков в данных? Запишите названия признаков, имеющих пропуски, и попробуйте для любых двух из них дать обоснование, почему их значения могут быть пропущены.

In [4]:
features_count = features.count()
nan = len(features) - features_count
print(nan[nan != 0])

first_blood_time               19553
first_blood_team               19553
first_blood_player1            19553
first_blood_player2            43987
radiant_bottle_time            15691
radiant_courier_time             692
radiant_flying_courier_time    27479
radiant_first_ward_time         1836
dire_bottle_time               16143
dire_courier_time                676
dire_flying_courier_time       26098
dire_first_ward_time            1826
dtype: int64


Обоснование для пропусков:  
first_blood_time - ни одна команда не совершила "первую кровь" в первые 5 минут игры;  
first_blood_team - пропуск, т.к. ни одна команда не совершила "первую кровь";  
first_blood_player1 - пропуск, т.к. ни один игрок не совершил "первую кровь";  
first_blood_player2 - пропуск, т.к. ни один игрок не помог совершить "первую кровь";  
radiant_bottle_time - команда radiant не приобрела предмет "bottle" в течение первых 5 минут;  
radiant_courier_time - команда radiant не приобрела предмет "courier" в течение первых 5 минут;  
radiant_flying_courier_time - команда radiant не приобрела предмет "flying_courier" в течение первых 5 минут;  
radiant_first_ward_time - команда radiant не установила ни одного наблюдателя в течение первых 5 минут;  
dire_bottle_time - команда dire не приобрела предмет "bottle" в течение первых 5 минут;  
dire_courier_time - команда dire не приобрела предмет "courier" в течение первых 5 минут;  
dire_flying_courier_time - команда dire не приобрела предмет "flying_courier" в течение первых 5 минут;  
dire_first_ward_time - команда dire не установила ни одного наблюдателя в течение первых 5 минут.

3. Замените пропуски на нули с помощью функции fillna(). На самом деле этот способ является предпочтительным для логистической регрессии, поскольку он позволит пропущенному значению не вносить никакого вклада в предсказание. Для деревьев часто лучшим вариантом оказывается замена пропуска на очень большое или очень маленькое значение — в этом случае при построении разбиения вершины можно будет отправить объекты с пропусками в отдельную ветвь дерева. Также есть и другие подходы — например, замена пропуска на среднее значение признака. Мы не требуем этого в задании, но при желании попробуйте разные подходы к обработке пропусков и сравните их между собой.

In [5]:
features = features.fillna(0)

4. Какой столбец содержит целевую переменную? Запишите его название.  
Целевая переменная - radiant_win.  

5. Забудем, что в выборке есть категориальные признаки, и попробуем обучить градиентный бустинг над деревьями на имеющейся матрице "объекты-признаки". Зафиксируйте генератор разбиений для кросс-валидации по 5 блокам (KFold), не забудьте перемешать при этом выборку (shuffle=True), поскольку данные в таблице отсортированы по времени, и без перемешивания можно столкнуться с нежелательными эффектами при оценивании качества. Оцените качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации, попробуйте при этом разное количество деревьев (как минимум протестируйте следующие значения для количества деревьев: 10, 20, 30). Долго ли настраивались классификаторы? Достигнут ли оптимум на испытанных значениях параметра n_estimators, или же качество, скорее всего, продолжит расти при дальнейшем его увеличении?

In [164]:
kf = KFold(n_splits = 5, shuffle = True)

trees = [10, 20, 30]
scores = {}

for tree in trees:
    print("Number of trees: ", tree)
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=tree)
    scores[tree] = np.mean(cross_val_score(estimator = clf, X = features, y = y, cv = kf, scoring = 'roc_auc'))
    print(scores[tree])
    print("Time elapsed: ", datetime.datetime.now() - start_time, "\n")

Number of trees:  10
0.6652894619669059
Time elapsed:  0:00:24.442177 

Number of trees:  20
0.6831820693892156
Time elapsed:  0:00:42.490265 

Number of trees:  30
0.6893840512326603
Time elapsed:  0:01:04.906526 



По мере увеличения числа деревьев возрастает значение метрик AUC-ROC, поэтому имеет смысл использовать еще больше деревьев.   
Для ускорения процесса обучения можно уменьшить размер выборки для обучения или уменьшить глубину деревьев.  

## Подход 2: логистическая регрессия

1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C). Какое наилучшее качество у вас получилось? Как оно соотносится с качеством градиентного бустинга? Чем вы можете объяснить эту разницу? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

Отмасштабируем признаки:

In [7]:
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

Подберем параметр C.

In [166]:
grid = {'C': np.power(10.0, np.arange(-5, 6, 1))}
clf2 = LogisticRegression(random_state = 1, verbose = 0)
gs = GridSearchCV(clf2, grid, cv = kf, scoring = 'roc_auc')
gs.fit(features_scaled, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=1, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='roc_auc', verbose=0)

In [15]:
print(gs.best_estimator_)
print(gs.best_score_)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.7163051677377854


Лучший параметр регуляризации (C): 0.01   
Полученное значение (0.716) выше значения, полученного с использованием градиентного бустинга. Это может быть связано с тем, что в методе градиентного бустинга использовалось малое количество деревьев.  
С использованием логистической регрессии модель обучается быстрее, чем с градиентным бустингом.

2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [16]:
drop_columns2 = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero',\
                 'd3_hero', 'd4_hero', 'd5_hero']
features_new = features.drop(drop_columns2, axis='columns')
features_new.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,5,2098,1489,20,0,0,7,3,842,...,0,35.0,103.0,-84.0,221.0,3,4,2,2,-52.0
1,1430220345,4,1188,1033,9,0,1,12,4,1596,...,0,-20.0,149.0,-84.0,195.0,5,4,3,1,-5.0
2,1430227081,4,1319,1270,22,0,0,12,3,1314,...,1,-39.0,45.0,-77.0,221.0,3,4,3,1,13.0
3,1430263531,4,1779,1056,14,0,0,5,2,539,...,0,-30.0,124.0,-80.0,184.0,0,4,2,0,27.0
4,1430282290,4,1431,1090,8,1,0,8,2,629,...,0,46.0,182.0,-80.0,225.0,6,3,3,0,-16.0


In [17]:
features_scaled_new = scaler.fit_transform(features_new)

In [18]:
grid = {'C': np.power(10.0, np.arange(-5, 6, 1))}
clf2 = LogisticRegression(random_state = 1, verbose = 0)
gs = GridSearchCV(clf2, grid, cv = kf, scoring = 'roc_auc')
gs.fit(features_scaled_new, y)
print(gs.best_estimator_)
print(gs.best_score_)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.7164580259224748


Качество практически не изменилось.  
Это может быть связано с тем, что категориальные признаки были неверно трактованы как числовые, и их вклад в классификацию оказался слишком мал.

3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [30]:
heroes = []
heroes_columns = ['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero',\
                  'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
for hero in heroes_columns:
    heroes.append(np.unique(features[hero]))
    
print(np.unique(heroes))
print(len(np.unique(heroes)))

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55
  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73
  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91
  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 109 110 112]
108


4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

Из предыдущего пункта видно, что всего в игре 112 героев.

In [36]:
# N — количество различных героев в выборке
N = 112
X_pick = np.zeros((features.shape[0], N))

for i, match_id in enumerate(features.index):
    for p in range(5):
        X_pick[i, features.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

X_pick

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., -1.]])

In [37]:
X = np.concatenate((features_scaled_new, X_pick), axis = 1)
print(X)

[[-2.54436416  1.40080818  1.52597175 ...  0.          0.
   0.        ]
 [-2.54045236  0.50131354 -0.08013929 ...  0.          0.
   0.        ]
 [-2.53923104  0.50131354  0.1510701  ...  0.          0.
   0.        ]
 ...
 [ 1.09874571  0.50131354  0.29226667 ...  0.          0.
   1.        ]
 [ 1.09895204 -0.39818111 -0.17368203 ...  0.          0.
   0.        ]
 [ 1.1026479  -0.39818111 -0.31840851 ...  0.          0.
  -1.        ]]


5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [39]:
grid = {'C': np.power(10.0, np.arange(-5, 6, 1))}
clf2 = LogisticRegression(random_state = 1, verbose = 0)
gs = GridSearchCV(clf2, grid, scoring='roc_auc', cv=kf)
gs.fit(X, y)
print(gs.best_estimator_)
print(gs.best_score_)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=1, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)
0.7518900069648286


Лучший параметр регуляризации (C): 0.1  
Полученное качество (0.752) больше предыдущего результата. Это может быть объяснено тем, что теперь каждый конкретный герой кодируется отдельной переменной и между идентификаторами героев отсутствуют количественные и порядковые связи.

6. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [154]:
X_test = pd.read_csv('./features_test.csv', index_col='match_id')

In [155]:
X_test = X_test.fillna(0)

In [156]:
X_pick_test = np.zeros((X_test.shape[0], N))

for i, match_id in enumerate(X_test.index):
    for p in range(5):
        X_pick_test[i, X_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_test[i, X_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [157]:
X_test.drop(drop_columns2, inplace=True, axis='columns')

In [158]:
X_test = scaler.fit_transform(X_test)

In [159]:
X_test = np.concatenate((X_test, X_pick_test), axis = 1)

In [160]:
y_test = gs.best_estimator_.predict_proba(X_test)

In [161]:
y_test = pd.DataFrame(y_test)

In [162]:
y_test.head()

,0,1
0,0.175335,0.824665
1,0.242835,0.757165
2,0.812214,0.187786
3,0.137514,0.862486
4,0.760083,0.239917


In [163]:
print('Minimal: ', min(y_test[1]))
print('Maximal: ', max(y_test[1]))

Minimal:  0.00858062348851582
Maximal:  0.9964586846326406


Можно убедиться, что предсказанные вероятности находятся на отрезке [0, 1] и не совпадают между собой.